In [1]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.mongoexp import MongoTrials
import numpy as np

X = np.load(open("X_all.npy", "rb"))
y = np.load(open("y_all.npy", "rb"))
X_unknown = np.load(open("X_unknown_all.npy", "rb"))

Scale the data and preserver the zeros

In [2]:
from sklearn.preprocessing import MaxAbsScaler

scaler = MaxAbsScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_unknown_scaled = scaler.transform(X_unknown)

In [3]:
space = { 'gb_loss': hp.choice('gb_loss', [{'deviance': 'deviance'}, {'exponential':'expoential'}]),
           'learning_rate': hp.uniform('learning_rate', 0.01, 0.05),
          'X': X_scaled, 'y': y, 'n_folds': 2 }

def objective(space):   
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.cross_validation import KFold
    from sklearn import metrics
    import numpy as np
    import time
    
    gb_loss = space['gb_loss']
    learning_rate = space['learning_rate']
    X = space['X']
    y = space['y']
    n_folds = space['n_folds']
    kfolds = KFold(n=len(X), n_folds=n_folds, shuffle=True, random_state=int(time.time()))
    scores = []
    for train, test in kfolds:
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
        clf = GradientBoostingClassifier(loss=gb_loss, learning_rate=learning_rate)
        clf.fit(X_train, y_train)
        pred = clf.predict_proba(X_test)
        fpr, tpr, thresholds = metrics.roc_curve(y_test, pred[:,1])
        scores.append(metrics.auc(fpr, tpr))
    output = { 'status': STATUS_OK, 'loss': 1 - np.mean(scores), 'loss_variance': np.var(scores) }
    print(output)
    return output

# trials = MongoTrials('mongo://hyperopt@ds011840.mlab.com:11840/santander-hyperopt/jobs', exp_key='exp1')
trials = Trials()
%time best = fmin(objective, space=space, algo=tpe.suggest, max_evals=2, trials=trials)

print(best)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
for trial in trials:
    print(trial)